# Mathematics Architecture for RocketeerCore

Made by Peter Go
Last Update Pushed to Repo: Oct. 23rd

This notebook explains and simulates, along with the explanations, the underlying mathematics behind the Rocketeer simulation software. This notebook usually explains the significance of the topic being coded into the code block below, then uses the equation to run and graph an updated version of the simulation. This can be done as many times as wanted, adding and adding to the existing simulation while keeping the parts separate and making things look, and making things easier to actually implement. Not to mention the easiness of updating the simulation to another version in in CI/CD workflow.

The basic and fundamental parts of an aerodynamics simulation that the notebook will look at include, but are not limited to:

- Model Entry
- Drag
- Lift (Fin Induced, Not really needed)
- Thrust

## Rocket Types Used and Implemented Successfully

- <input type="checkbox" checked /> Mini
- <input type="checkbox" /> Standard


## Mathematics Physics Elements Implemented Successfully

- <input type="checkbox" checked /> Entry Derived Calculations
- <input type="checkbox" checked /> Basic Thrust
  - <input type="checkbox"  /> Thrust Curve Equations
  - <input type="checkbox"  /> Noise of Thrust Curve
- <input type="checkbox" checked /> Gravity
- <input type="checkbox" checked /> Drag Equation
- <input type="checkbox" /> Lift Equation (sparse, not targeted)

## Simple ~~Dynamic~~ Array Thrust

> Dynamic thrust is not going to be defined in the current version of the simulation. 
> Instead, we are going to be using arrays to define thrust and be as precise as possible using available data.

Thrust is defined using npArrays and the simulation uses the thrust given for the current time period to calculate the thrust applied onto the Rocket's pointing axis.

In [24]:
import numpy as np
import matplotlib

gravityGlobalSL = 9.807

# Thrust Data contributed by John Coker from thrustcurve.org 
def thrustArrayThrowback(motor):
    if motor == "C5" or motor == "c5":
        return np.array([[0.00, 0.00][0.0050, 1.422], [0.031, 2.695], [0.063, 5.503], [0.117, 10.444], [0.151, 14.374], [0.168, 16.358], [0.174, 16.471], [0.179, 17.594], [0.199, 18.604], [0.211, 19.877], [0.217, 20.401], [0.23, 20.289], [0.234, 18.679], [0.239, 18.23], [0.252, 13.813], [0.258, 9.021], [0.265, 6.588], [0.28, 4.904], [0.296, 4.08], [0.349, 3.481], [0.438, 2.995], [0.504, 2.733], [0.611, 2.807], [0.815, 2.695], [0.97, 2.695], [1.335, 2.695], [1.598, 2.658], [1.794, 2.695], [1.96, 2.733], [1.987, 1.16], [1.992, 0.0]])

<>:9: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
<>:9: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
/var/folders/k6/rprfxmhd0v55y8fdl1vp6vsw0000gn/T/ipykernel_6711/1406463346.py:9: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
  return np.array([[0.00, 0.00][0.0050, 1.422], [0.031, 2.695], [0.063, 5.503], [0.117, 10.444], [0.151, 14.374], [0.168, 16.358], [0.174, 16.471], [0.179, 17.594], [0.199, 18.604], [0.211, 19.877], [0.217, 20.401], [0.23, 20.289], [0.234, 18.679], [0.239, 18.23], [0.252, 13.813], [0.258, 9.021], [0.265, 6.588], [0.28, 4.904], [0.296, 4.08], [0.349, 3.481], [0.438, 2.995], [0.504, 2.733], [0.611, 2.807], [0.815, 2.695], [0.97, 2.695], [1.335, 2.695], [1.598, 2.658], [1.794, 2.695], [1.96, 2.733], [1.987, 1.16], [1.992, 0.0]])


## Defining the Rocket

This rocket model is a common **Conic** nose cone shape and a **Cylindrical** body. Since it is a mini rocket, it does not have fins to stabilize and performs the most basic and minimized requirements for the rocket simulation, and does not push the calculations to any limit, and proves the bare minimum.

In [25]:
# Class definition of a rocket. This will be expanded in the future to include parameters that are more "defining" of a rocket and be changed to a 3D version of a rocket.
class Rocket:
    def __init__(self, calculated, rocketType, motor, thrustArray, diameter, noseconeLength, bodyLength, weight, frontalArea, cD, invincible ):
        self.rocketType = rocketType
        self.calculated = calculated
        self.motor = motor
        self.thrustArray = thrustArray
        self.diameter = diameter
        self.noseconeLength = noseconeLength
        self.bodyLength = bodyLength
        self.weight = weight
        self.frontalArea = frontalArea
        self.cD = cD
        self.invincible = invincible
    
    # This method (required to be called, or the simulation doesn't run) calculates the parts of the Rocket that are not given to the simulation. Of course, it won't calculate simple things or impossible things, like cD
    def calculateNonBased(self):
        if self.rocketType == "Mini":
            self.cD = 0.2
            self.frontalArea = 2 * (np.pi) * (self.diameter/2) 
            self.weight = self.weight / 1000 * gravityGlobalSL

        self.thrustArray = np.array([[0.00, 0.00][0.0050, 1.422], [0.031, 2.695], [0.063, 5.503], [0.117, 10.444], [0.151, 14.374], [0.168, 16.358], [0.174, 16.471], [0.179, 17.594], [0.199, 18.604], [0.211, 19.877], [0.217, 20.401], [0.23, 20.289], [0.234, 18.679], [0.239, 18.23], [0.252, 13.813], [0.258, 9.021], [0.265, 6.588], [0.28, 4.904], [0.296, 4.08], [0.349, 3.481], [0.438, 2.995], [0.504, 2.733], [0.611, 2.807], [0.815, 2.695], [0.97, 2.695], [1.335, 2.695], [1.598, 2.658], [1.794, 2.695], [1.96, 2.733], [1.987, 1.16], [1.992, 0.0]])

<>:23: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
<>:23: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
/var/folders/k6/rprfxmhd0v55y8fdl1vp6vsw0000gn/T/ipykernel_6711/3523538035.py:23: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
  self.thrustArray = np.array([[0.00, 0.00][0.0050, 1.422], [0.031, 2.695], [0.063, 5.503], [0.117, 10.444], [0.151, 14.374], [0.168, 16.358], [0.174, 16.471], [0.179, 17.594], [0.199, 18.604], [0.211, 19.877], [0.217, 20.401], [0.23, 20.289], [0.234, 18.679], [0.239, 18.23], [0.252, 13.813], [0.258, 9.021], [0.265, 6.588], [0.28, 4.904], [0.296, 4.08], [0.349, 3.481], [0.438, 2.995], [0.504, 2.733], [0.611, 2.807], [0.815, 2.695], [0.97, 2.695], [1.335, 2.695], [1.598, 2.658], [1.794, 2.695], [1.96, 2.733], [1.987, 1.16], [1.992, 0.0]])


In [29]:
# Making a Rocket
localRocket = Rocket(False, "Mini", "C5", np.array([]), 5, 10, 20, 50, 0, 0, False)
Rocket.calculateNonBased(localRocket)

TypeError: list indices must be integers or slices, not tuple

## Simulating!!!

In [ ]:
simulationRuntimeFlag = True
print(f"✓ Passed Flag 1!")
print(f"{localRocket.thrustArray[(1,1)]}")
timeStep = 0


while simulationRuntimeFlag == True:
    # Acceleration of localRocket
    dragForce = 

    # Ending...
    timeStep += 0.01
    simulationRuntimeFlag = False